In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1,2'
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'


In [2]:
import sys
sys.path.append('../')
from utils import load_data
import pandas as pd
import numpy as np
np.random.seed(42)

In [3]:
from tqdm import tqdm
import torch

In [4]:
settings = 'cross_file_first'
data = load_data('test', 'r', 'python', settings)
raw_samples = data['easy']
len(raw_samples)
raw_samples = np.random.choice(raw_samples, 3000)

Loading data: 100%|██████████| 1/1 [00:01<00:00,  1.32s/it]


In [6]:
from llama import Llama

In [7]:
root = "/home/sir_timio/workspace/codellama"
version = "CodeLlama-7b"

In [8]:
ckpt_dir = f"{root}/{version}"
tokenizer_path = f"{root}/{version}/tokenizer.model"
temperature = 0.2
top_p = 0.9
max_seq_len = 256
max_batch_size = 4
max_gen_len = None

In [9]:
import torch.distributed as dist

# Manually setting environment variables for distributed training
os.environ['MASTER_ADDR'] = 'localhost'
os.environ['MASTER_PORT'] = '12355'
os.environ['RANK'] = '0'
os.environ['WORLD_SIZE'] = '1'

dist.init_process_group(backend='nccl', rank=int(os.environ['RANK']), world_size=int(os.environ['WORLD_SIZE']))

dist.destroy_process_group()

In [10]:
if not torch.distributed.is_initialized():
    torch.distributed.init_process_group("nccl") 

In [12]:
generator = Llama.build(
    ckpt_dir=ckpt_dir,
    tokenizer_path=tokenizer_path,
    max_seq_len=max_seq_len,
    max_batch_size=max_batch_size,
    model_parallel_size=1,
)

AssertionError: Loading a checkpoint for MP=1 but world size is 3